# Analyzer: Retail Sales

Yasmin Johana Garcia
Yesid Humberto Montaño

In [ ]:
import os
# indicate your Spark version, here we use Spark 3.5 with pydeequ 1.4.0
os.environ["SPARK_VERSION"] = '3.2'
os.environ["HADOOP_HOME"] = "C:\\hadoop\\winutils\\hadoop-3.3.6"

In [ ]:
from pyspark.sql import SparkSession, Row, DataFrame
import json
import pandas as pd
import sagemaker_pyspark

import pydeequ

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

In [ ]:
df = spark.read.parquet("Retail_sales.parquet")
df.printSchema()

root
 |-- Store ID: string (nullable = true)
 |-- Product ID: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- Units Sold: long (nullable = true)
 |-- Sales Revenue (USD): double (nullable = true)
 |-- Discount Percentage: long (nullable = true)
 |-- Marketing Spend (USD): long (nullable = true)
 |-- Store Location: string (nullable = true)
 |-- Product Category: string (nullable = true)
 |-- Day of the Week: string (nullable = true)
 |-- Holiday Effect: boolean (nullable = true)



In [ ]:
df.columns

['Store ID',
 'Product ID',
 'Date',
 'Units Sold',
 'Sales Revenue (USD)',
 'Discount Percentage',
 'Marketing Spend (USD)',
 'Store Location',
 'Product Category',
 'Day of the Week',
 'Holiday Effect']

In [ ]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("Store ID")) \
                    .addAnalyzer(ApproxCountDistinct("Product ID")) \
                    .addAnalyzer(Mean("Units Sold")) \
                    .addAnalyzer(Correlation("Sales Revenue (USD)", "Store Location")) \
                    .addAnalyzer(Correlation("Sales Revenue (USD)", "Product Category")) \
                    .addAnalyzer(Correlation("Sales Revenue (USD)", "Discount Percentage")) \
                    .addAnalyzer(Correlation("Sales Revenue (USD)", "Marketing Spend (USD)")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-----------+--------------------+-------------------+--------------------+
|     entity|            instance|               name|               value|
+-----------+--------------------+-------------------+--------------------+
|     Column|          Units Sold|               Mean|   6.161966666666666|
|Multicolumn|Sales Revenue (US...|        Correlation|-0.00273208445823...|
|     Column|            Store ID|       Completeness|                 1.0|
|     Column|          Product ID|ApproxCountDistinct|                44.0|
|Multicolumn|Sales Revenue (US...|        Correlation|-0.06579056019654646|
|    Dataset|                   *|               Size|             30000.0|
+-----------+--------------------+-------------------+--------------------+



C:\Users\Victus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [ ]:
analysisResult_pandas_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult, pandas=True)
analysisResult_pandas_df

C:\Users\Victus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


,entity,instance,name,value
0,Column,Units Sold,Mean,6.161967
1,Multicolumn,"Sales Revenue (USD),Marketing Spend (USD)",Correlation,-0.002732
2,Column,Store ID,Completeness,1.000000
3,Column,Product ID,ApproxCountDistinct,44.000000
4,Multicolumn,"Sales Revenue (USD),Discount Percentage",Correlation,-0.065791
5,Dataset,*,Size,30000.000000
